In [7]:
!python --version

Python 3.10.12


In [8]:
!pip install transformers datasets torch

In [9]:
from datasets import load_dataset

In [10]:
dataset = load_dataset('SKNahin/bengali-transliteration-data')

In [11]:
print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})
{'bn': 'স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???', 'rm': 'scroll kore 20/30 second er video pann nai???'}


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('t5-small')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [13]:
def preprocess_function(examples):
    inputs = examples['rm']
    targets = examples['bn']
    model_inputs = tokenizer(inputs, text_target=targets, truncation=True)
    return model_inputs

In [14]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5006 [00:00<?, ? examples/s]

In [15]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer

model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [17]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False,
)

In [19]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Assume a 80/20 train/test split
train_test_dataset = tokenized_datasets['train'].train_test_split(test_size=0.2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_dataset['train'],  # Use the training split from train_test_dataset
    eval_dataset=train_test_dataset['test'],  # Use the test split from train_test_dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-19-d113c354b2b7>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.436300,0.228577
2,0.262600,0.195224
3,0.242000,0.184861


TrainOutput(global_step=1503, training_loss=0.3134237859380777, metrics={'train_runtime': 5041.1568, 'train_samples_per_second': 2.383, 'train_steps_per_second': 0.298, 'total_flos': 120292427563008.0, 'train_loss': 0.3134237859380777, 'epoch': 3.0})

In [21]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.1848612278699875, 'eval_runtime': 115.1537, 'eval_samples_per_second': 8.701, 'eval_steps_per_second': 1.094, 'epoch': 3.0}


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
# Google Drive to save Model
save_path = '/content/drive/MyDrive/BitFest-Preli-Model'

# Save the model
trainer.save_model(save_path)


In [39]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Define the path to your saved model
model_path = '/content/drive/MyDrive/BitFest-Preli-Model'

# Load the tokenizer and model from the specified path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


In [ ]:
import torch

def transliterate(input_text):
    """
    Transliterates the given input text using the loaded model and tokenizer.

    Args:
        input_text: The input text to transliterate.

    Returns:
        The transliterated text.
    """
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generate output using the model
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)  # Adjust max_new_tokens if needed

    # Decode the generated tokens back to text
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text

In [5]:
# Get predictions using the predict method
predictions = trainer.predict(trainer.eval_dataset)

# Access predicted labels (adjust argmax if necessary based on your model output)
predicted_labels = predictions.predictions.argmax(-1)

# Extract reference labels
reference_labels = predictions.label_ids

# Calculate accuracy using predicted_labels and reference_labels
accuracy = compute_accuracy(predicted_labels, reference_labels)

print(f"Accuracy: {accuracy}")

NameError: name 'trainer' is not defined